In [1]:
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from config import *
from typing import Tuple

from data.util import split_weekdays_and_weekends

from prediction_models.dbn import *
from prediction_models.kelm import *

%reload_ext autoreload
%autoreload 2

In [2]:
DBN_HIDDEN_LAYER_SIZES = CONFIG['DBN_HIDDEN_LAYER_SIZES']
GIBBS_SAMPLING_STEPS = CONFIG['GIBBS_SAMPLING_STEPS']
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)
TRAIN_START_DATE = datetime.strptime(CONFIG['TRAIN_START_DATE'], DATE_FORMAT)
TRAIN_END_DATE = datetime.strptime(CONFIG['TRAIN_END_DATE'], DATE_FORMAT)
TRAIN_TIME_WINDOW_SIZE = CONFIG['TRAIN_TIME_WINDOW_SIZE']

### Prepare training dataset

Load the matrix $C$ constructed from $Q$

In [3]:
mat_c = torch.load(out_path('mat_c.pt'))
TIME_DIM, SPACE_DIM = mat_c.shape
mat_c.shape

torch.Size([2880, 12])

Split $C$ into workdays and weekends data

In [4]:
mat_c_wd, mat_c_we = split_weekdays_and_weekends(mat_c, TRAIN_START_DATE, TRAIN_END_DATE)
assert mat_c_wd.shape[1] == mat_c_we.shape[1] == mat_c.shape[1]
assert mat_c_wd.shape[0] + mat_c_we.shape[0] == mat_c.shape[0]
mat_c_wd.shape, mat_c_we.shape

(torch.Size([2112, 12]), torch.Size([768, 12]))

Create a dataset for pre-training RBMs

In [6]:

class BinaryVectorDataset(Dataset):
    def __init__(self, n_bits: int):
        self.n_bits = n_bits
        self.format_str = f'{{0:0{self.n_bits}b}}'
    
    def __len__(self) -> int:
        return 2 ** self.n_bits
    
    def __getitem__(self, index: int) \
            -> Tuple[torch.TensorType, torch.TensorType]:
        bin_str = self.format_str.format(index)
        return torch.tensor([float(c) for c in bin_str]), torch.tensor([index,])

dbn_pre_train_loader = DataLoader(BinaryVectorDataset(SPACE_DIM), batch_size=256)

### Pre-train RBMs inside DBN

Train to reconstruct all possible combinations of 0's and 1's

In [7]:
dbn = DBN(SPACE_DIM, DBN_HIDDEN_LAYER_SIZES, k=GIBBS_SAMPLING_STEPS)
pre_train_dbn(dbn, dbn_pre_train_loader, print_every=1)

Epoch 0, Machine 0:	Loss: 3.63346529006958
Epoch 0, Machine 1:	Loss: 10.110261917114258
Epoch 0, Machine 2:	Loss: 6.241438865661621
Epoch 1, Machine 0:	Loss: 0.27469363808631897
Epoch 1, Machine 1:	Loss: 3.0161426067352295
Epoch 1, Machine 2:	Loss: 1.9354207515716553
Epoch 2, Machine 0:	Loss: -0.944818377494812
Epoch 2, Machine 1:	Loss: 0.374968558549881
Epoch 2, Machine 2:	Loss: 0.4830241799354553
Epoch 3, Machine 0:	Loss: -1.3790874481201172
Epoch 3, Machine 1:	Loss: -0.6266959309577942
Epoch 3, Machine 2:	Loss: -0.038316428661346436
Epoch 4, Machine 0:	Loss: -1.6058722734451294
Epoch 4, Machine 1:	Loss: -0.7914646863937378
Epoch 4, Machine 2:	Loss: -0.46987029910087585
Epoch 5, Machine 0:	Loss: -1.7674893140792847
Epoch 5, Machine 1:	Loss: -0.8008990287780762
Epoch 5, Machine 2:	Loss: -0.6826756596565247
Epoch 6, Machine 0:	Loss: -1.876910924911499
Epoch 6, Machine 1:	Loss: -0.7868847846984863
Epoch 6, Machine 2:	Loss: -0.7667257785797119
Epoch 7, Machine 0:	Loss: -1.959479212760925

DBN(
  (rbms): ModuleList(
    (0-2): 3 x RBM()
  )
)

#### Train prediction of the first section

Define the loss function

In [8]:
loss_fn = nn.MSELoss()

In [9]:
kelm = KELM()
for start_time in range(TIME_DIM - TRAIN_TIME_WINDOW_SIZE - 1):
    end_time = start_time + TRAIN_TIME_WINDOW_SIZE
    train_window = mat_c[start_time:end_time] # [60, 12]
    label_window = mat_c[start_time + 1:end_time + 1][:, 0] # [60]
    features = dbn(train_window) # [60, 12]
    kelm.fit(features, label_window)
    predictions = kelm(features) # [60, 60]
    print(predictions.shape)
    print(loss_fn(predictions, label_window))
    # USE THIS TO TRAIN DBN

RuntimeError: batch2 must be a 3D tensor